<a href="https://colab.research.google.com/github/legiando/DataScienceCourse/blob/master/Spark_Streaming_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark Streaming Laboratory



## Spark Bootstraping for Google Colab

Run this before start working with the notebooks from the spark course. 
When you will start a new (and fresh) notebook at Colab. Google Cloud will create a new Docker container just for your use. 

Executing this notebook will install into the container the software. The container will be reused by the user until it will destroy by inactivity.


In [0]:
!apt-get install openjdk-8-jdk-headless netcat -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark 

## Set Environment Variables
Set the locations where Spark and Java are installed.

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] pyspark-shell"

## Cloning our Github repo

In [0]:
!rm -rf /content/SparkCourse2019/
!git clone https://github.com/bazarum/SparkCourse2019.git

## Start a SparkSession
This will start a local Spark session.

In [0]:
import findspark
print(findspark.init())

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
# from pyspark import SparkContext
# sc = SparkContext(master = 'local[2]')


## We will need Tweepy 

In [0]:
!pip install -q tweepy

## This will be our tweets extractor


In [0]:
from __future__ import absolute_import, print_function

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

consumer_key = 'w8TL68073oA9fBB8uY4jmXQ1C'
consumer_secret = 'mWkBQcUKu9MFUAGSVOUjAv6EIKHmEwOLoohVYXdTEoq8glcKO1'
access_token = '870541-Xxzh1Oke7Y4VqWxU6mqjO441TPTJ0xwmE3IjAYjXsLJ'
access_token_secret = 'INTa3rPviJQ0AhAhK4vEceuTo2OkBxAK00f9J7YrWYEd7'

class StdOutListener(StreamListener):
    """ A listener handles tweets that are received from the stream.
    This is a basic listener that just prints received tweets to stdout.

    """
    def on_data(self, data):
        print(data)
        return True

    def on_error(self, status):
        print(status)

if __name__ == '__main__':
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    stream = Stream(auth, l)
    stream.filter(track=['selfie'])


## Now we can start getting some insights

In [0]:
 df = spark.read.json('/content/SparkCourse2019/datasets/tweets/tweets.json', )

In [0]:

df.show() 
df.printSchema()

In [0]:
from pyspark.sql import functions as f
result = df.select(f.col("text"), f.col("entities.media.media_url"))
result2 = df.select(["text", "entities.media.media_url"])
result.show(10)
result.printSchema()
print(result.collect())

In [0]:
df.createOrReplaceTempView("tweets")
result3 = spark.sql("SELECT text, entities.hashtags.text as hashtags, entities.media.media_url[0] as picture FROM tweets")
result3.show(50)

In [0]:
from pyspark.sql.functions import explode
allhashes = result3.withColumn("hashtag", explode(result3.hashtags))

In [0]:
allhashes.createOrReplaceTempView("hashtags")
hashes = spark.sql("SELECT hashtag, count(*) as tot FROM hashtags GROUP BY hashtag ORDER BY tot DESC LIMIT 20")
hashes.show(50)

## Extra ball: how terminal access to the colab

In [0]:
#Generate root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))



In [0]:
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip


In [0]:
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')



In [0]:
#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('export USER=root && ./ngrok authtoken $authtoken')
get_ipython().system_raw('export USER=root && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))


In [0]:
! export USER=root && ./ngrok authtoken 7ta2zQwaM7BNPiDYpacrd_6gNx9yPBvpA7dy3DTc7NJ && ./ngrok tcp 22 &

In [0]:
!# export USER=root && ./ngrok tcp 22 &
!ps ax | grep ngrok
!kill -9 1771

In [0]:
#Get public address
! curl -s http://localhost:4040/api/tunnels  # | python3 -c \
     # "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Extra Ball 2: How to connect your drive 

In [0]:
from google.colab import drive
drive.mount('./drive')

In [0]:
!ls -lt "drive/My Drive" | head -10

## Streaming from a Socket

In [0]:
from pyspark.sql.functions import from_json, split


tweets = spark.readStream.format("socket").option("host", "0.tcp.ngrok.io").option("port", 14439).load()

In [0]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "0.tcp.ngrok.io") \
    .option("port", 14439) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

In [0]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

In [0]:
from pyspark.sql.functions import from_json

data = spark.read.json(tweets.rdd.map(lambda r: r.json))

# data = tweets.select(from_json(tweets.value).alias("dat"))


data.printSchema()


# total = data.groupBy("line").count()

query = data.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

In [0]:
import findspark

findspark.init('/Users/donghua/spark-2.3.2-bin-hadoop2.7')

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

# Not required if running in Pyspark integrated notebooksc = SparkContext()

ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

socket_stream = ssc.socketTextStream("127.0.0.1",5555)


lines = socket_stream.window(20)

from collections import namedtuple
fields = ("tag","count")
Tweet = namedtuple('Tweet', fields)

# use () for multiple lines(lines.flatMap(lambda text: text.split(" "))
.filter(lambda word: word.startswith("#"))
.map(lambda word: (word.lower(), 1))
.reduceByKey(lambda a, b : a + b)
.map(lambda rec: Tweet(rec[0],rec[1]))
.foreachRDD(lambda rdd: rdd.toDF().sort(desc("count"))
.limit(10).registerTempTable("tweets")))


import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
# get_ipython().run_line_magic('matplotlib', 'inline')

ssc.start()

count = 0
while count < 10:
    time.sleep(10)
    top_10_tweets = sqlContext.sql("select tag,count from tweets")
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait = True)
    plt.figure(figsize= (10, 8))
    sns.barplot(x="count", y="tag",data=top_10_df)
    plt.show()
    count = count+1
ssc.stop()


In [0]:
squery = spark.readStream.format("rate").load


da

# data = squery.writeStream.format("console").start()
                                                    
# data.awaitTermination()
